In [1]:
sys.path.append("../")
from config import config
from feature_generation import get_all_atms_feature_set
from preprocessing import get_input_sets, scaler_fit_transform, scaler_transform, scaler_inverse_transform
from tabTransformer import TabTransformer
from misc import nmae_error, load_pickle

import optuna
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Load Data

In [3]:
load_config = config['load_config']

try:
    config = read_hyperparameters_from_file(load_config['hyperparameter_path'])
except:
    print("WARNING: Hyperparameter file (%s) not found. Using the default config." % load_config['hyperparameter_path'])

clusters = load_config['clusters']

df = pd.read_csv("../" + load_config['data_path'])
all_atms_feature_set = get_all_atms_feature_set(df, first_n = load_config['n_atms'])
all_atms_feature_set.sort_index(inplace = True)

# Reading Pickles
for cluster_feature in clusters:
    d = load_pickle("../" + clusters[cluster_feature]['path'])
    all_atms_feature_set[cluster_feature] = all_atms_feature_set['AtmId'].map(d)

## Setting Features

In [5]:
feature_config  = config['feature_config']

categorical_features = [cat for cat in
    all_atms_feature_set.select_dtypes(include=feature_config['categorical_column_types'])
    if cat not in feature_config['excluded_categorical']]
continuous_features = [cat for cat in
    all_atms_feature_set.select_dtypes(include=feature_config['continuous_column_types'])
    if cat not in feature_config['excluded_continuous']]

groups = [continuous_features]
groups.extend(categorical_features)

## Aranging train/test Data

In [6]:
X = all_atms_feature_set[continuous_features + categorical_features]
y = all_atms_feature_set[feature_config['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

# MinMaxTransform
X_train, y_train, scaler_X, scaler_y = scaler_fit_transform(X_train, y_train, continuous_features)
X_test, y_test = scaler_transform(X_test, y_test, scaler_X, scaler_y, continuous_features)

X_train = get_input_sets(X_train, groups)
X_test  = get_input_sets(X_test, groups)

..\preprocessing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = scaler_X.transform(X[numerical_features])
C:\Users\cahid\anaconda3\envs\tensor\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
..\preprocessing.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

## Optuna

In [7]:
def objective(trial):

    param = {
        'dim': trial.suggest_categorical('dim', [2,4,8]),
        'depth': trial.suggest_categorical('depth', [1,2,3,6,12]),
        'heads': trial.suggest_categorical('heads', [2,4,8]),
        'attn_dropout': trial.suggest_categorical('attn_dropout', [0,0.1,0.2,0.3,0.4,0.5]),
        'ff_dropout': trial.suggest_categorical('ff_dropout', [0,0.1,0.2,0.3,0.4,0.5])
    }

    mlp_activation = trial.suggest_categorical('mlp_activation', ['relu', 'selu'])
    mlp_hidden = [
        (trial.suggest_categorical('mlp_1_dim', [256, 128, 64]), mlp_activation),
        (trial.suggest_categorical('mlp_2_dim', [64, 32, 16]), mlp_activation),
    ]

    tabTransformer = TabTransformer(
        categories = [len(all_atms_feature_set[cat].unique())
        if cat not in clusters.keys() else
        clusters[cat]['n_clusters']
        for cat in categorical_features],
        num_continuous = len(continuous_features),
        dim_out=1,
        mlp_hidden=mlp_hidden,
        **param
    )

    training_config = config['training_config']

    tabTransformer.compile(
        optimizer = tf.optimizers.Adam(learning_rate = trial.suggest_loguniform('learning_rate', 0.005, 0.5)),
        loss = training_config['loss']
    )

    tabTransformer.fit(
        X_train,
        y_train,
        epochs = 5,
        batch_size = training_config['batch_size'],
        verbose = False)

    error = nmae_error(scaler_inverse_transform(y_test, scaler_y), scaler_y.inverse_transform(tabTransformer.predict(X_test)))
    print(error)

    return error

In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)


print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

[I 2021-08-16 12:56:30,489] A new study created in memory with name: no-name-2ae91358-c80f-4032-9878-d62e589bd4af
[I 2021-08-16 12:58:32,083] Trial 0 finished with value: 0.4960148772084476 and parameters: {'dim': 8, 'depth': 6, 'heads': 4, 'attn_dropout': 0, 'ff_dropout': 0.3, 'mlp_activation': 'relu', 'mlp_1_dim': 128, 'mlp_2_dim': 32, 'learning_rate': 0.027794021760071407}. Best is trial 0 with value: 0.4960148772084476.


0.4960148772084476


[I 2021-08-16 12:58:57,956] Trial 1 finished with value: 0.4528914646101807 and parameters: {'dim': 4, 'depth': 2, 'heads': 4, 'attn_dropout': 0.4, 'ff_dropout': 0, 'mlp_activation': 'selu', 'mlp_1_dim': 256, 'mlp_2_dim': 64, 'learning_rate': 0.01979513325467289}. Best is trial 1 with value: 0.4528914646101807.


0.4528914646101807


[I 2021-08-16 12:59:19,050] Trial 2 finished with value: 0.4960526883650339 and parameters: {'dim': 8, 'depth': 2, 'heads': 2, 'attn_dropout': 0.4, 'ff_dropout': 0.4, 'mlp_activation': 'relu', 'mlp_1_dim': 256, 'mlp_2_dim': 16, 'learning_rate': 0.014691151008679938}. Best is trial 1 with value: 0.4528914646101807.


...

0.29787149271741253


[I 2021-08-16 13:25:52,981] Trial 18 finished with value: 0.42871598965763874 and parameters: {'dim': 2, 'depth': 6, 'heads': 8, 'attn_dropout': 0.5, 'ff_dropout': 0.1, 'mlp_activation': 'selu', 'mlp_1_dim': 64, 'mlp_2_dim': 64, 'learning_rate': 0.005770561330846152}. Best is trial 11 with value: 0.2969554478414453.


0.42871598965763874


[I 2021-08-16 13:26:51,559] Trial 19 finished with value: 0.4282102126921229 and parameters: {'dim': 8, 'depth': 2, 'heads': 8, 'attn_dropout': 0.2, 'ff_dropout': 0, 'mlp_activation': 'relu', 'mlp_1_dim': 64, 'mlp_2_dim': 16, 'learning_rate': 0.13805196600484013}. Best is trial 11 with value: 0.2969554478414453.


0.4282102126921229
Number of finished trials: 20
Best trial: {'dim': 4, 'depth': 6, 'heads': 8, 'attn_dropout': 0.1, 'ff_dropout': 0.5, 'mlp_activation': 'relu', 'mlp_1_dim': 64, 'mlp_2_dim': 32, 'learning_rate': 0.0052039954462104075}
Best value: 0.2969554478414453


In [9]:
import json
params = study.best_trial.params
with open('../hyperparameters.txt', 'w') as outfile:
    json.dump(params, outfile)

In [10]:
params

{'dim': 4,
 'depth': 6,
 'heads': 8,
 'attn_dropout': 0.1,
 'ff_dropout': 0.5,
 'mlp_activation': 'relu',
 'mlp_1_dim': 64,
 'mlp_2_dim': 32,
 'learning_rate': 0.0052039954462104075}